In [ ]:
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("wandb_api")
wandb.login(key=api_key);

wandb.init(project='ML4SCI_Pytorch', entity='himanshu_nitrr')


In [ ]:
hf_e = h5py.File('../input/ml4sci/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5', 'r')
electron_data_x = hf_e.get('X')
electron_data_y = hf_e.get('y')

In [ ]:
hf_p = h5py.File('../input/ml4sci/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5', 'r')
photon_data_x = hf_p.get('X')
photon_data_y = hf_p.get('y')

In [ ]:
electron_data_x = np.asarray(electron_data_x)
electron_data_y = np.asarray(electron_data_y)

photon_data_x = np.asarray(photon_data_x)
photon_data_y = np.asarray(photon_data_y)

In [ ]:
print(electron_data_x.shape)
print(photon_data_x.shape)

print(electron_data_y.shape)
print(photon_data_y.shape)

In [ ]:
X = np.concatenate((electron_data_x, photon_data_x))
y = np.concatenate((electron_data_y, photon_data_y))

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_vis = X[:5]

fig, axes = plt.subplots(nrows=len(X_vis), ncols=2, figsize=(48, 48))
for i in range(len(X_vis)):

    axes[i][0].imshow(X_vis[i,:,:,0])
    axes[i][1].imshow(X_vis[i,:,:,1])



In [ ]:
print(X.shape)

In [ ]:
print(X[0,:,:,1])

In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)
X, y = shuffle(X, y, random_state=0) #really random 

In [ ]:
Config = {
    'MODEL_NAME': "vgg11",
    'N_CLASSES': 1,  #binary classification
    'TRAIN_BS': 32,
    'VALID_BS': 32,
    'N_EPOCHS': 100,
    'NUM_WORKERS': 2,
    'LR': 1e-3,
    'DEVICE': 'cuda',
    'scheduler':'CosineAnnealingLR',
    'min_lr':1e-6,
    'batch_size':32,
    'weight_decay':1e-6,
    'T_max':3,
}

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import Adam
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
# mean = 0.015
# std = 0.297

In [ ]:
X_train = X[:496900]
X_val = X[496900:497900]
X_test = X[497900:]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
y_train = y[:496900]
y_val = y[496900:497900]
y_test = y[497900:]

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import Adam
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

In [ ]:
tensor_x_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)

train_dataset = TensorDataset(tensor_x_train,tensor_y_train) 
train_dataloader = DataLoader(train_dataset, 
                                batch_size=Config['TRAIN_BS'],
                                shuffle=True,
                                pin_memory=True,
                                num_workers=Config['NUM_WORKERS'],                                
                             ) 

In [ ]:
tensor_x_val = torch.Tensor(X_val)
tensor_y_val = torch.Tensor(y_val)

val_dataset = TensorDataset(tensor_x_val,tensor_y_val) 
val_dataloader = DataLoader(val_dataset,
                            batch_size=Config['VALID_BS'],
                            shuffle=False,
                            num_workers=Config['NUM_WORKERS'],
                           ) 

In [ ]:
tensor_x_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)

test_dataset = TensorDataset(tensor_x_test,tensor_y_test) 

test_dataloader = DataLoader(test_dataset,batch_size=Config['VALID_BS'],
                            shuffle=False,
                            num_workers=Config['NUM_WORKERS'],
                            ) 

In [ ]:
class CreateModel(nn.Module):
    def __init__(self, model_name = Config['MODEL_NAME'], pretrained = False):
        super().__init__()

        self.model_name = model_name
        self.head = timm.create_model(self.model_name, pretrained = pretrained, num_classes = Config['N_CLASSES'], in_chans=2)

    def forward(self, x):
        x = self.head(x)
        return x

In [ ]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
        
#         # Convolution 1
#         self.cnn1 = nn.Conv2d(in_channels=2, out_channels=16, kernel_size=5, stride=1, padding=0)
#         self.relu1 = nn.ReLU()
        
#         # Max pool 1
#         self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
#         # Convolution 2
#         self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
#         self.relu2 = nn.ReLU()
        
#         # Max pool 2
#         self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
#         # Fully connected 1
#         self.fc1 = nn.Linear(800, Config['N_CLASSES']) 
    
#     def forward(self, x):
#         # Convolution 1
#         out = self.cnn1(x)
#         out = self.relu1(out)
        
#         # Max pool 1
#         out = self.maxpool1(out)
        
#         # Convolution 2 
#         out = self.cnn2(out)
#         out = self.relu2(out)
        
#         # Max pool 2 
#         out = self.maxpool2(out)
        
#         # flatten
#         out = out.view(out.size(0), -1)

#         # Linear function (readout)
#         out = self.fc1(out)
        
#         return out

In [ ]:
model = CreateModel()
model.to(Config['DEVICE'])

In [ ]:
# model = Net()
# model.to(Config['DEVICE'])

In [ ]:
img_size = 32

In [ ]:
VAL_LOSS = []
TRAIN_LOSS = []

VAL_ROC_AUC = []
TRAIN_ROC_AUC = []

In [ ]:
def get_scheduler(optimizer):
    scheduler = CosineAnnealingLR(optimizer, T_max=Config['T_max'], eta_min=Config['min_lr'], last_epoch=-1)
    return scheduler

In [ ]:
def val_fn(loader, model, loss_fn):
    VAL_PROBA = []
    VAL_TARGETS = []
    model.eval()
    loop = tqdm(loader)

    val_loss = 0
    roc_auc = 0
    for batch_idx, (data, targets) in enumerate(loop):
        data = F.normalize(data)
        
        data = data.to(device='cuda')
        data = data.reshape(-1, 2, img_size, img_size)
        
        targets = targets.to(device='cuda')
        targets = targets.unsqueeze(1)
#         VAL_TARGETS.append(targets.detach().cpu().numpy())
        VAL_TARGETS = np.concatenate((VAL_TARGETS, targets.detach().cpu().numpy().reshape(-1)))
#         targets = F.one_hot(targets, num_classes=Config['N_CLASSES'])
        
        # forward
        with torch.cuda.amp.autocast():
            with torch.no_grad():
                logits = model(data)
                proba = torch.sigmoid(logits)
#                 VAL_PROBA.append(proba.detach().cpu().numpy())
                VAL_PROBA = np.concatenate((VAL_PROBA, proba.detach().cpu().numpy().reshape(-1)))
                loss = loss_fn(logits, targets)
                preds = (proba > 0.5).float()
#                 roc_auc_b = roc_auc_score(targets.detach().cpu().numpy(), proba.detach().cpu().numpy())
#                 roc_auc += roc_auc_b
#                 print(loss)
        val_loss += loss.item()

        wandb.log({"val_loss": loss})
                
            # loss = sm.losses.DiceLos

        # update tqdm loop
        loop.set_postfix(val_loss=loss.item())
        
    
    VAL_TARGETS = VAL_TARGETS.reshape(-1)
    VAL_PROBA = VAL_PROBA.reshape(-1)
    roc_auc = roc_auc_score(VAL_TARGETS, VAL_PROBA)
    wandb.log({"val_roc_auc": roc_auc})
#     roc_auc /= len(loader)
    VAL_ROC_AUC.append(roc_auc)
    print(f"Validation roc_auc_score: {roc_auc}")
    val_loss /= len(loader)
    model.train()
    VAL_LOSS.append(val_loss)
    return val_loss

In [ ]:
def train_fn(loader, val_loader, model, optimizer, scheduler, loss_fn):
    TRAIN_PROBA = []
    TRAIN_TARGETS = []
    model.train()
    loop = tqdm(loader)
    roc_auc = 0
    train_loss = 0
    
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device='cuda')
        data = data.reshape(-1, 2, img_size, img_size)

        targets = targets.to(device='cuda')
        targets = targets.unsqueeze(1)
#         TRAIN_TARGETS.append(targets.detach().cpu().numpy())
        TRAIN_TARGETS = np.concatenate((TRAIN_TARGETS, targets.detach().cpu().numpy().reshape(-1)))
        with torch.cuda.amp.autocast():
#             print(torch.max(data))
            data = F.normalize(data)
#             print(torch.max(data))
            logits = model(data)
            proba = torch.sigmoid(logits)
#             TRAIN_PROBA.append(proba.detach().cpu().numpy())
            TRAIN_PROBA = np.concatenate((TRAIN_PROBA, proba.detach().cpu().numpy().reshape(-1)))
            preds = (proba > 0.5).float()
            loss = loss_fn(logits, targets)

#             roc_auc_b = roc_auc_score(targets.detach().cpu().numpy(), proba.detach().cpu().numpy())
#             roc_auc += roc_auc_b

        train_loss += loss.item()

        wandb.log({"train_loss": loss})
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print(type(TRAIN_TARGETS))

        # update tqdm loop
        loop.set_postfix(loss=loss.item())
        
#     roc_auc /= len(loader)
    
    TRAIN_TARGETS = TRAIN_TARGETS.reshape(-1)
    TRAIN_PROBA = TRAIN_PROBA.reshape(-1)
    
    roc_auc = roc_auc_score(TRAIN_TARGETS, TRAIN_PROBA)
    TRAIN_ROC_AUC.append(roc_auc)
    wandb.log({"train_roc_auc": roc_auc})
    
    print(f"Training roc_auc_score: {roc_auc}")
    train_loss /= len(loader)
    TRAIN_LOSS.append(train_loss)
    val_loss = val_fn(val_loader, model, loss_fn)
    return val_loss

In [ ]:
MAX_PATIENCE = 20

optimizer = Adam(model.parameters(), lr=Config['LR'], weight_decay=Config['weight_decay'], amsgrad=False)
scheduler = get_scheduler(optimizer)

loss_fn = nn.BCEWithLogitsLoss()
flag = -1
patience = 0
wandb.watch(model)
for epoch in range(Config['N_EPOCHS']):
    print(f"{'='*20} Epoch: {epoch+1} {'='*20}")
    curr_val_loss = train_fn(train_dataloader, val_dataloader, model, optimizer, scheduler, loss_fn)
    scheduler.step()
    
    if flag == -1:
        flag = 1
        best_loss = curr_val_loss
    if curr_val_loss < best_loss:
        best_loss = curr_val_loss
        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        torch.save(checkpoint, 'best_weights.pth.tar')
    elif curr_val_loss > best_loss:
        patience += 1
    if patience > MAX_PATIENCE:
        print(f"patience exceeded than {MAX_PATIENCE}")
        break
# save model

In [ ]:
## PLOT TRAIN AND VAL LOSS
plt.plot(TRAIN_LOSS, color='red')
plt.plot(VAL_LOSS, color='blue')
plt.show()

In [ ]:
## PLOT TRAIN AND VAL ROC_AUC_SCORE
plt.plot(TRAIN_ROC_AUC, color='red')
plt.plot(VAL_ROC_AUC, color='blue')
plt.show()